# 🧠 Machine Learning Pipeline for Clinical Outcome Prediction

This notebook implements the full machine-learning pipeline used in our study to predict three clinical outcomes:

- **In-hospital Mortality**
- **ICU Admission**
- **Length of Stay (LOS)**

It is organised into clearly defined stages covering data inputs, cross-validation setup, feature set construction, model training, and output generation.

---

##  Input Files (Human-Readable Description)

The pipeline uses the following data matrices and corresponding feature name lists:

| Variable         | Description |
|------------------|-------------|
| `X_clin`         | Clinical variables (e.g., age, vitals, comorbidities) |
| `clinical_names` | Feature names for clinical variables |
| `X_amrvir`       | Antimicrobial resistance & virulence gene features |
| `amrvir_names`   | AMR/Virulence feature names |
| `X_pangenome`    | Pangenome presence/absence features |
| `pangenome_names`| Pangenome feature names |
| `X_snps`         | Core genome SNP features |
| `snps_names`     | SNP feature names |
| `X_unitigs`      | Unitig sequence features |
| `unitigs_names`  | Unitig feature names |

---

## PHASE A Creating Cross-Validation Folds

We generate **five outer cross-validation folds** of train, validation, and test splits.

- The indices for each fold are saved and reused across all models to ensure consistent comparison.
- Each fold is used for model training, selection, and held-out evaluation.

---

## PHASE B Feature Set Configurations

Different combinations of data layers are defined via **postfix keys**.  
To switch the prediction target, replace the postfix suffix `Mortality` with:

- `ICU` → ICU Admission  
- `LOS` → Length of Stay

```python
postfix_data = {
    "_1_Mortality": ([X_clin], clinical_names),

    "_2_Mortality": ([X_clin, X_amrvir],
                     clinical_names + amrvir_names),

    "_3_Mortality": ([X_clin, X_amrvir, X_pangenome],
                     clinical_names + amrvir_names + pangenome_names),

    "_4_Mortality": ([X_clin, X_amrvir, X_pangenome, X_snps],
                     clinical_names + amrvir_names + pangenome_names + snps_names),

    "_5_Mortality": ([X_clin, X_amrvir, X_pangenome, X_snps, X_unitigs],
                     clinical_names + amrvir_names + pangenome_names + snps_names + unitigs_names),

    "_6_Mortality": ([X_amrvir, X_pangenome, X_snps, X_unitigs],
                     amrvir_names + pangenome_names + snps_names + unitigs_names)
}
```

## PHASE C Machine Learning Training Overview

This section details which algorithms are trained for each outcome type.

---

#### Binary Outcomes: Mortality & ICU Admission

For Mortality and ICU Admission prediction, the following classifiers are trained:

• **Elastic Net Logistic Regression**  
Trained on each fold and feature set; balances L1 and L2 penalties for feature selection and regularisation.

• **XGBoost Classifier**  
Trained on each fold and feature set; gradient-boosted trees that model non-linear relationships.  
Feature importance analysis is also provided for XGBoost using metrics such as gain and/or SHAP values.

✔ Both Elastic Net and XGBoost classifiers are trained for Mortality and ICU tasks.  
✔ XGBoost training includes downstream feature importance analysis.

---

#### Continuous Outcome: Length of Stay (LOS)

For LOS regression, the following models are trained:

• **Quantile Regressor**  
Estimates conditional quantiles of LOS (e.g., median and tails) to capture distributional effects.

• **NGBoost**  
A probabilistic gradient boosting model that produces predictive distributions and uncertainty estimates.

✔ Both Quantile Regression and NGBoost are trained for LOS prediction.


## PHASE A — Save Train/Test Indices

The following script generates 5 non-stratified folds for a continuous outcome using `KFold` from scikit-learn. For each outer fold, a random 15 % subset of the training indices is also held out for internal validation. All fold indices along with feature names are saved as compressed `.npz` files. A splits manifest JSON records split parameters.

In [ ]:
import json
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

# =========================
# SAVE NON-OVERLAPPING CV SPLITS (classification)
# =========================
# Required in memory:
#   X : np.ndarray (n_samples, n_features)
#   y : np.ndarray (n_samples,)   # binary or categorical target
#   feature_names : list[str]

# ------------------------------------------------------
# Project root (portable)
# ------------------------------------------------------
BASE_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

folds_dir = BASE_DIR / "saved_folds_Mortality"
folds_dir.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------
# Parameters
# ------------------------------------------------------
N_FOLDS      = 5
RANDOM_STATE = 42

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# ------------------------------------------------------
# Create and save fold indices
# ------------------------------------------------------
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):

    np.savez_compressed(
        folds_dir / f"fold{fold}_idx.npz",
        train_idx=train_idx,
        test_idx=test_idx,
        feature_names=np.asarray(feature_names, dtype=object)
    )

# ------------------------------------------------------
# Manifest file
# ------------------------------------------------------
with open(folds_dir / "splits_manifest.json", "w") as f:
    json.dump({
        "n_samples"      : int(X.shape[0]),
        "n_features"     : int(X.shape[1]),
        "n_folds"        : N_FOLDS,
        "target_type"    : "classification",
        "stratification" : True,
        "validation_set" : False
    }, f, indent=2)

print(f"Saved {N_FOLDS} non-overlapping folds in: {folds_dir.resolve()}")

## PHASE B — Create input dataframe for the predictive classifiers 

The following script generates input predictive data and label data for the the two nominal labels, i.e. **Moratlity** (Death) and **ICU Admission**. 

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

BASE_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
DATA_DIR = BASE_DIR
postfix = "_3_Mortality" # or _2_Morality based on the desired input
# ------------------------
# Load datasets
# ------------------------
df_X = pd.read_csv(DATA_DIR / "Metadata_predictor_Mortality.csv")
df_amrvir = pd.read_csv(DATA_DIR / "AMRVIR_predictor_Mortality.csv")
df_pangenome = pd.read_csv(DATA_DIR / "Pangenome_predictor_Mortality.csv")
df_snps = pd.read_csv(DATA_DIR / "SNPs_predictor_Mortality.csv")
df_unitigs = pd.read_csv(DATA_DIR / "Unitigs_predictor_Mortality.csv")
df_pheno = pd.read_csv(DATA_DIR / "df_phenotype_Mortality.csv")

# ------------------------
# Merge once (perfect alignment)
# ------------------------
df = (
    df_X
    .merge(df_amrvir, on=ID_COL)
    .merge(df_pangenome, on=ID_COL)
    .merge(df_snps, on=ID_COL)
    .merge(df_unitigs, on=ID_COL)
    .merge(df_pheno[[ID_COL, OUTCOME_COL]], on=ID_COL)
)

y = df[OUTCOME_COL].astype(int).to_numpy()

# ------------------------
# Clinical preprocessing
# ------------------------
feature_cols_num = ["CHARLSON","AGE","BMI"]
feature_cols_cat = ["GENDER","SOURCE","RGN"]

preprocess = ColumnTransformer([
    ("num", Pipeline([("imp",SimpleImputer(strategy="median")),
                      ("sc",StandardScaler())]), feature_cols_num),
    ("cat", Pipeline([("imp",SimpleImputer(strategy="most_frequent")),
                      ("ohe",OneHotEncoder(handle_unknown="ignore"))]), feature_cols_cat)
])

X_clin = preprocess.fit_transform(df[feature_cols_num + feature_cols_cat])
cat_names = preprocess.named_transformers_["cat"].named_steps["ohe"].get_feature_names_out(feature_cols_cat)
clinical_names = list(feature_cols_num) + list(cat_names)
clinical_names = [f"{c}_clinical" for c in clinical_names]

# ------------------------
# Omics blocks
# ------------------------
def block(df_source, suffix):
    cols = [c for c in df_source.columns if c != ID_COL]
    return df[cols].to_numpy(), [f"{c}_{suffix}" for c in cols]

X_amrvir, amrvir_names = block(df_amrvir, "amrvir")
X_pangenome, pangenome_names = block(df_pangenome, "pangenome")
X_snps, snps_names = block(df_snps, "snps")
X_unitigs, unitigs_names = block(df_unitigs, "unitigs")

# ------------------------
# Postfix → feature map
# ------------------------
FEATURE_MAP = {
    "_1_Mortality": ([X_clin], clinical_names),
    "_2_Mortality": ([X_clin, X_amrvir], clinical_names + amrvir_names),
    "_3_Mortality": ([X_clin, X_amrvir, X_pangenome],
                     clinical_names + amrvir_names + pangenome_names),
    "_4_Mortality": ([X_clin, X_amrvir, X_pangenome, X_snps],
                     clinical_names + amrvir_names + pangenome_names + snps_names),
    "_5_Mortality": ([X_clin, X_amrvir, X_pangenome, X_snps, X_unitigs],
                     clinical_names + amrvir_names + pangenome_names + snps_names + unitigs_names),
    "_6_Mortality": ([X_amrvir, X_pangenome, X_snps, X_unitigs]
}

# ------------------------
# Build X
# ------------------------
blocks, feature_names = FEATURE_MAP[postfix]
X = np.hstack(blocks).astype(np.float32)

df = pd.read_csv(DATA_DIR / "df_phenotype_Mortality.csv")
# Assign labels (y) from the Death column
y = df["Mortality"].values

#For LOS make this log1p transformation beforehand
y =  y.astype(np.int32)       
y =  np.log1p(y)

print("postfix:", postfix)
print("X shape:", X.shape)
print("y shape:", y.shape)

# PHASE C /A/Classifers - Elastic-Net Logistic Regression with nested CV

- Expects X, y, feature_names in memory
- Uses saved train/test indices: saved_folds/fold{n}_idx.npz
- Performs 3-fold inner CV for hyperparameter tuning
- Saves models, metrics (train/val/test), and OOF predictions

In [ ]:
import os
import itertools
import joblib
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.model_selection import StratifiedKFold

# --- Config ---
base        = Path("saved_folds").parent
folds_dir   = base / "saved_folds"
models_dir  = base / f"saved_models_lr{postfix}"
preds_dir   = base / f"preds_lr{postfix}"
metrics_dir = base / f"metrics_lr{postfix}"
inner_val_dir = metrics_dir / "inner_val"

for d in [models_dir, preds_dir, metrics_dir, inner_val_dir]:
    d.mkdir(parents=True, exist_ok=True)

n_splits_outer = 5
n_splits_inner = 3

grid_lr = {
    "C": [0.1, 1.0],
    "l1_ratio": [0.0, 0.5, 1.0],
}

def make_metrics(y_true, y_prob, thr=0.5):
    y_hat = (y_prob >= thr).astype(int)
    return {
        "auc":  roc_auc_score(y_true, y_prob),
        "acc":  accuracy_score(y_true, y_hat),
        "prec": precision_score(y_true, y_hat, zero_division=0),
        "rec":  recall_score(y_true, y_hat, zero_division=0),
        "f1":   f1_score(y_true, y_hat, zero_division=0),
    }

# --- Nested CV ---
all_rows = []
oof_preds = np.zeros(len(y))
oof_true  = np.zeros(len(y), dtype=int)

for fold in range(1, n_splits_outer + 1):
    idx = np.load(folds_dir / f"fold{fold}_idx.npz", allow_pickle=True)
    train_idx, test_idx = idx["train_idx"], idx["test_idx"]

    X_tr, y_tr = X[train_idx], y[train_idx]
    X_te, y_te = X[test_idx], y[test_idx]

    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)
    best_auc, best_params = -np.inf, None

    for C, l1r in itertools.product(grid_lr["C"], grid_lr["l1_ratio"]):
        aucs_inner, val_metrics = [], []

        for inner_i, (it, iv) in enumerate(inner_cv.split(X_tr, y_tr), start=1):
            sub_idx = train_idx[it]
            val_idx = train_idx[iv]

            pipe = Pipeline([
                ("scaler", StandardScaler()),
                ("lr", LogisticRegression(
                    max_iter=5000,
                    solver="saga",
                    penalty="elasticnet",
                    C=C,
                    l1_ratio=l1r,
                    random_state=42
                ))
            ])
            pipe.fit(X[sub_idx], y[sub_idx])
            val_probs = pipe.predict_proba(X[val_idx])[:, 1]

            m = make_metrics(y[val_idx], val_probs)
            m.update({"outer_fold": fold, "inner_fold": inner_i, "C": C, "l1_ratio": l1r})
            val_metrics.append(m)
            aucs_inner.append(m["auc"])

        pd.DataFrame(val_metrics).to_csv(
            inner_val_dir / f"lr_inner_val_fold{fold}_C{C}_l1{l1r}{postfix}.csv",
            index=False
        )

        mean_auc = np.mean(aucs_inner)
        if mean_auc > best_auc:
            best_auc = mean_auc
            best_params = {"C": C, "l1_ratio": l1r}

    # Train final model
    final_model = Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(
            max_iter=5000,
            solver="saga",
            **best_params,
            random_state=42
        ))
    ])
    final_model.fit(X_tr, y_tr)

    joblib.dump(final_model, models_dir / f"lr_fold{fold}{postfix}.joblib")
    with open(models_dir / f"lr_fold{fold}{postfix}_params.json", "w") as f:
        json.dump(best_params, f)

    # Predict
    train_probs = final_model.predict_proba(X_tr)[:, 1]
    test_probs  = final_model.predict_proba(X_te)[:, 1]

    all_rows.append({"set":"train","fold":fold,**make_metrics(y_tr, train_probs)})
    all_rows.append({"set":"test","fold":fold,**make_metrics(y_te, test_probs)})

    pd.DataFrame({
        "index": test_idx,
        "fold": fold,
        "y_true": y_te,
        "y_score": test_probs
    }).to_csv(preds_dir / f"lr_test_fold{fold}{postfix}.csv", index=False)

    oof_preds[test_idx] = test_probs
    oof_true[test_idx]  = y_te

pd.DataFrame(all_rows).to_csv(metrics_dir / f"lr_metrics_all_folds{postfix}.csv", index=False)
pd.DataFrame({"index":np.arange(len(y)),"y_true":oof_true,"y_score":oof_preds}).to_csv(
    preds_dir / f"lr_oof{postfix}.csv", index=False
)

# PHASE C /A/Classifers - XGBoost Classifer with nested CV

- Expects X, y, feature_names in memory
- Uses saved train/test indices: saved_folds/fold{n}_idx.npz
- Performs 3-fold inner CV for hyperparameter tuning
- Saves models, metrics (train/val/test), and OOF predictions


In [ ]:
import os
import itertools
import joblib
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# --- Config ---
base        = Path("saved_folds").parent
folds_dir   = base / "saved_folds"
models_dir  = base / f"saved_models_xgb{postfix}"
preds_dir   = base / f"preds_xgb{postfix}"
metrics_dir = base / f"metrics_xgb{postfix}"
inner_val_dir = metrics_dir / "inner_val"

for d in [models_dir, preds_dir, metrics_dir, inner_val_dir]:
    d.mkdir(parents=True, exist_ok=True)

n_splits_outer = 5
n_splits_inner = 3

grid_xgb = {
    "learning_rate": [0.03],
    "max_depth": [2,3],
    "subsample": [0.8,1.0],
    "colsample_bytree": [1.0]
}

def make_metrics(y_true, p_prob, thr=0.5):
    y_hat = (p_prob >= thr).astype(int)
    return {
        "auc":  roc_auc_score(y_true, p_prob),
        "acc":  accuracy_score(y_true, y_hat),
        "prec": precision_score(y_true, y_hat, zero_division=0),
        "rec":  recall_score(y_true, y_hat, zero_division=0),
        "f1":   f1_score(y_true, y_hat, zero_division=0),
    }

# --- Nested CV ---
all_rows = []
oof_preds = np.zeros(len(y))
oof_true  = np.zeros(len(y), dtype=int)

for fold in range(1, n_splits_outer+1):
    idx = np.load(folds_dir / f"fold{fold}_idx.npz", allow_pickle=True)
    train_idx, test_idx = idx["train_idx"], idx["test_idx"]

    best_auc, best_params = -np.inf, None

    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)
    for lr, md, ss, cs in itertools.product(
        grid_xgb["learning_rate"],
        grid_xgb["max_depth"],
        grid_xgb["subsample"],
        grid_xgb["colsample_bytree"]
    ):
        aucs_inner = []
        val_records = []

        for i, (it, iv) in enumerate(inner_cv.split(X[train_idx],y[train_idx]),start=1):
            sub_idx = train_idx[it]
            val_idx = train_idx[iv]

            model = XGBClassifier(
                objective="binary:logistic",
                eval_metric="auc",
                tree_method="hist",
                random_state=42,
                n_jobs=-1,
                n_estimators=2000,
                learning_rate=lr,
                max_depth=md,
                subsample=ss,
                colsample_bytree=cs
            )
            model.fit(X[sub_idx], y[sub_idx], verbose=False)
            val_probs = model.predict_proba(X[val_idx])[:,1]

            m = make_metrics(y[val_idx], val_probs)
            m.update({"outer_fold":fold,"inner_fold":i,"lr":lr,"md":md,"ss":ss,"cs":cs})
            val_records.append(m)
            aucs_inner.append(m["auc"])

        pd.DataFrame(val_records).to_csv(
            inner_val_dir / f"xgb_inner_val_fold{fold}_lr{lr}_md{md}_ss{ss}_cs{cs}{postfix}.csv",
            index=False
        )

        mean_auc = np.mean(aucs_inner)
        if mean_auc > best_auc:
            best_auc = mean_auc
            best_params = {"learning_rate": lr, "max_depth": md, "subsample": ss, "colsample_bytree": cs}

    # Train final XGB model
    final_model = XGBClassifier(
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        random_state=42,
        n_jobs=-1,
        n_estimators=2000,
        **best_params
    )
    final_model.fit(X[train_idx], y[train_idx], verbose=False)

    joblib.dump(final_model, models_dir / f"xgb_fold{fold}{postfix}.joblib")
    with open(models_dir / f"xgb_fold{fold}{postfix}_params.json", "w") as f:
        json.dump(best_params, f)

    # Predictions
    train_probs = final_model.predict_proba(X[train_idx])[:,1]
    test_probs  = final_model.predict_proba(X[test_idx])[:,1]

    all_rows.append({"set":"train","fold":fold,**make_metrics(y[train_idx],train_probs)})
    all_rows.append({"set":"test","fold":fold,**make_metrics(y[test_idx], test_probs)})

    pd.DataFrame({
        "index": test_idx,
        "fold": fold,
        "y_true": y[test_idx],
        "y_score": test_probs
    }).to_csv(preds_dir / f"xgb_test_fold{fold}{postfix}.csv", index=False)

    oof_preds[test_idx] = test_probs
    oof_true[test_idx]  = y[test_idx]

# Aggregate
pd.DataFrame(all_rows).to_csv(metrics_dir / f"xgb_metrics_all_folds{postfix}.csv",index=False)
pd.DataFrame({"index":np.arange(len(y)),"y_true":oof_true,"y_score":oof_preds}).to_csv(
    preds_dir / f"xgb_oof{postfix}.csv", index=False
)

# PHASE D /A/Classifers -SHAP Interpretation of XGBoost Models

This notebook computes SHAP (SHapley Additive exPlanations) values for XGBoost models that were previously trained using fixed outer cross-validation splits.  

### Objectives

- Compute SHAP values **only on held-out test sets** for each outer CV fold.
- Quantify:
  - Mean absolute SHAP importance per feature.
  - Mean signed SHAP contribution per feature.
  - Correlation between feature value and SHAP attribution.
- Aggregate SHAP statistics across folds and calculate 95% confidence intervals.
- Export:
  - Per-fold SHAP importance tables.
  - A final aggregated SHAP importance table.

### Inputs (already in memory)

- `X`: numpy array `(n_samples × n_features)`
- `y`: numpy array `(n_samples,)`
- `feature_names`: list/array of feature names
- `postfix`: experiment label (e.g. `_2`)

### Required files on disk

- Saved models:  
  `saved_models{postfix}/xgb_fold{fold}{postfix}.joblib`
- Fixed CV indices:  
  `saved_folds/fold{fold}_idx.npz`

### Outputs

- Per-fold SHAP tables:  
  `shap_xgb{postfix}/xgb_shap_importance_fold{fold}{postfix}.csv`
- Aggregated SHAP summary:  
  `shap_xgb{postfix}/xgb_shap_importance_agg{postfix}.csv`

This approach ensures **no data leakage**, since SHAP values are computed only on unseen test data.

In [ ]:
import warnings, joblib
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import t
import shap
from xgboost import XGBClassifier

# ------------------------------------------------------
# Project root (portable, no hard-coded paths)
# ------------------------------------------------------
BASE_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

folds_dir  = BASE_DIR / "saved_folds"
models_dir = BASE_DIR / f"saved_models{postfix}"
shap_dir   = BASE_DIR / f"shap_xgb{postfix}"
shap_dir.mkdir(parents=True, exist_ok=True)

N_FOLDS = 5

# ------------------------------------------------------
# Sanity checks
# ------------------------------------------------------
assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray)
feature_names = np.asarray(feature_names, dtype=object)
assert X.shape[1] == len(feature_names)

# ------------------------------------------------------
# Helper functions
# ------------------------------------------------------
def ci95(vals):
    vals = np.asarray(vals, float)
    n = np.isfinite(vals).sum()
    if n <= 1:
        return np.nanmean(vals), np.nan, np.nan, np.nan
    m, s = np.nanmean(vals), np.nanstd(vals, ddof=1)
    h = t.ppf(0.975, n-1) * s / np.sqrt(n)
    return m, s, m-h, m+h

def safe_corr(x, y):
    if np.allclose(x, x[0]) or np.allclose(y, y[0]):
        return np.nan
    return np.corrcoef(x, y)[0,1]

# ------------------------------------------------------
# Main SHAP loop
# ------------------------------------------------------
abs_map, sgn_map, cor_map = {}, {}, {}

for fold in range(1, N_FOLDS+1):
    print(f"Processing fold {fold}")
    idx = np.load(folds_dir / f"fold{fold}_idx.npz", allow_pickle=True)
    te_idx = idx["test_idx"]

    model = joblib.load(models_dir / f"xgb_fold{fold}{postfix}.joblib")
    explainer = shap.TreeExplainer(model)

    shap_vals = explainer.shap_values(X[te_idx])
    if isinstance(shap_vals, list):
        shap_vals = shap_vals[1]

    abs_m = np.abs(shap_vals).mean(0)
    sgn_m = shap_vals.mean(0)
    corrs = [safe_corr(X[te_idx,i], shap_vals[:,i]) for i in range(X.shape[1])]

    df_fold = pd.DataFrame({
        "feature": feature_names,
        "mean_abs_shap": abs_m,
        "mean_signed_shap": sgn_m,
        "corr_feat_shap": corrs
    }).sort_values("mean_abs_shap", ascending=False)

    df_fold.to_csv(shap_dir / f"xgb_shap_importance_fold{fold}{postfix}.csv", index=False)

    for f,a,s,c in zip(feature_names, abs_m, sgn_m, corrs):
        abs_map.setdefault(f,[]).append(a)
        sgn_map.setdefault(f,[]).append(s)
        cor_map.setdefault(f,[]).append(c)

# ------------------------------------------------------
# Aggregate across folds
# ------------------------------------------------------
rows = []
for f in feature_names:
    ma,sa,la,ha = ci95(abs_map[f])
    ms,ss,ls,hs = ci95(sgn_map[f])
    mc,sc,lc,hc = ci95(cor_map[f])

    direction = "uncertain"
    if lc > 0: direction="positive"
    elif hc < 0: direction="negative"

    rows.append([f,ma,sa,la,ha,ms,ss,ls,hs,mc,sc,lc,hc,direction])

agg = pd.DataFrame(rows, columns=[
    "feature","mean_abs","sd_abs","ci95_low_abs","ci95_high_abs",
    "mean_signed","sd_signed","ci95_low_signed","ci95_high_signed",
    "mean_corr","sd_corr","ci95_low_corr","ci95_high_corr","direction"
]).sort_values("mean_abs", ascending=False)

agg.to_csv(shap_dir / f"xgb_shap_importance_agg{postfix}.csv", index=False)

print("SHAP aggregation complete.")

# PHASE C /B/Regressor Nested Cross-Validation Workflow for Quantile Regression

This code implements a **nested cross-validation pipeline** to train and evaluate **Quantile Regression models** for predicting Length of Stay (LOS) with predictive intervals.

### What the Script Does

1. **Imports and Setup**  
   Loads required packages (e.g., NumPy, pandas, scikit-learn) and defines helper functions to compute metrics such as RMSE, Pearson and Spearman correlations, and basic regression metrics.

2. **Configuration**  
   - Sets random seed for reproducibility.
   - Defines paths for data, saved folds, models, metrics, and predictions.
   - Creates directories if they do not already exist.
   - Defines a hyperparameter grid for the Quantile Regressor.
   - Defines a list of quantiles (`0.05`, `0.50`, `0.95`) to train prediction intervals.

3. **Nested Cross-Validation Loop**
   - Loads precomputed outer CV fold indices (train/test splits).
   - For each outer fold:
     - Splits data into train, validation, and test sets.
     - Applies a log transform (`log1p`) to the target variable for stability.
     - Runs an **inner 3-fold CV loop** to select the best hyperparameters (based on median MAE in log scale).
     - Trains Quantile Regression models for each quantile (lower, median, upper) using the best hyperparameters found.
     - Evaluates models on the outer validation and test sets.
     - Computes performance metrics on both log and original scales (e.g., MAE, R², Spearman correlation).
     - Saves predictions, per-fold metrics, and models.

4. **Performance and Prediction Output**
   - Predicts for each quantile to form prediction intervals for LOS.
   - Saves test set predictions (median and interval bounds).
   - Saves nested CV metrics per fold (train/validation/test).
   - Computes and saves out-of-fold (OOF) predictions.
   - Outputs summary results and ends the script with a completion message.

### Structure

- **Outer Loop:** Iterates over saved folds.
- **Inner Loop:** Hyperparameter tuning via KFold.
- **Models Trained:**  
  Quantile Regression models for each quantile:  
  - `0.05` → Lower bound  
  - `0.50` → Median estimate  
  - `0.95` → Upper bound

### Evaluation Metrics

- **Original scale:** MAE, RMSE, R², Spearman correlation  
- **Interval statistics:** Coverage (10–90%) and mean interval width  
- **Log scale:** MAE, R² and correlation for log-transformed target

The code below carries out these steps and saves outputs into organized folders for models, metrics, and predictions, ready for post-hoc analysis and figure generation.

In [ ]:
#Nested Quantile Regression with Generic Paths

import os
import re
import glob
import itertools
import joblib
import numpy as np
import pandas as pd

from pathlib import Path
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import QuantileRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

# ================= Helper Functions =================

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def pearson_r(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0] if len(y_true) > 2 else np.nan

def spearman_r(y_true, y_pred):
    return spearmanr(y_true, y_pred)[0] if len(y_true) > 2 else np.nan

def metrics_basic(y_true, y_pred):
    return {
        "mae": mean_absolute_error(y_true, y_pred),
        "rmse": rmse(y_true, y_pred),
        "r2": r2_score(y_true, y_pred),
    }

# ================= Config =================

RNG = 42

# Use current working directory as base for outputs
base = Path.cwd()

folds_dir    = base / "saved_folds"
models_dir   = base / f"saved_models{postfix}"
metrics_dir  = base / f"metrics_qr{postfix}"
preds_dir    = base / f"preds_qr{postfix}"
inner_val_dir = metrics_dir / "inner_val"

# Create directories if needed
for d in (models_dir, metrics_dir, preds_dir, inner_val_dir):
    d.mkdir(parents=True, exist_ok=True)

# Hyperparameter grid for Quantile Regression
GRID = {
    "alpha": [1e-4, 1e-3, 1e-2, 0.05],
    "solver": ["highs"],
}

# Quantiles for building prediction intervals
QUANTILES = [0.05, 0.50, 0.95]

# Containers for aggregated results
all_rows = []
oof_preds = np.zeros(len(y))
oof_true  = np.zeros(len(y))

# Find saved fold index files
fold_files = sorted(
    glob.glob(str(folds_dir / "fold*_idx.npz")),
    key=lambda p: int(re.search(r"fold(\d+)_idx\.npz", os.path.basename(p)).group(1))
)

print("\n>>> Starting nested cross-validation for Quantile Regression ...")

# ================= Outer CV Loop =================

for fold_no, npz_path in enumerate(fold_files, start=1):
    print(f"\n=== Outer Fold {fold_no}/{len(fold_files)} ===")

    idx = np.load(npz_path, allow_pickle=True)
    tr_idx, te_idx = idx["train_idx"], idx["test_idx"]
    tr_sub_idx, val_idx = idx["train_sub_idx"], idx["val_idx"]

    # Slice data
    X_tr, y_tr = X[tr_idx], y[tr_idx].astype(float)
    X_te, y_te = X[te_idx], y[te_idx].astype(float)
    X_tr_sub, y_tr_sub = X[tr_sub_idx], y[tr_sub_idx].astype(float)
    X_val, y_val = X[val_idx], y[val_idx].astype(float)

    # Log-transform targets
    z_tr = np.log1p(y_tr)
    z_tr_sub = np.log1p(y_tr_sub)
    z_val = np.log1p(y_val)

    # ---------- Inner CV for Hyperparameter Tuning ----------

    inner_cv = KFold(n_splits=3, shuffle=True, random_state=RNG)
    best_val_mae_log = np.inf
    best_params = {}
    val_log = []

    val_pred_log_all = []
    val_true_log_all = []

    for alpha, solver in itertools.product(GRID["alpha"], GRID["solver"]):
        fold_maes = []
        for i, (inner_tr_i, inner_val_i) in enumerate(inner_cv.split(X_tr_sub), start=1):
            X_it, X_iv = X_tr_sub[inner_tr_i], X_tr_sub[inner_val_i]
            z_it, z_iv = z_tr_sub[inner_tr_i], z_tr_sub[inner_val_i]

            preds_val = {}
            for q in QUANTILES:
                qr = QuantileRegressor(quantile=q, alpha=alpha, solver=solver)
                qr.fit(X_it, z_it)
                preds_val[q] = qr.predict(X_iv)

            mae_val_log = mean_absolute_error(z_iv, preds_val[0.50])
            fold_maes.append(mae_val_log)

            val_log.append({
                "fold": fold_no,
                "inner_fold": i,
                "alpha": alpha,
                "solver": solver,
                "val_mae_log": mae_val_log
            })

            val_pred_log_all.extend(preds_val[0.50])
            val_true_log_all.extend(z_iv)

        mean_fold_mae_log = np.mean(fold_maes)
        if mean_fold_mae_log < best_val_mae_log:
            best_val_mae_log = mean_fold_mae_log
            best_params = {"alpha": alpha, "solver": solver}

    # Save inner validation results
    pd.DataFrame(val_log).to_csv(inner_val_dir / f"qr_inner_val_fold{fold_no}{postfix}.csv", index=False)

    print(f"Best params for fold {fold_no}: {best_params} | Inner MAE (log) = {best_val_mae_log:.4f}")

    # ---------- Fit Best Models and Save ----------

    best_models = {}
    for q in QUANTILES:
        qr = QuantileRegressor(
            quantile=q,
            alpha=best_params["alpha"],
            solver=best_params["solver"]
        )
        qr.fit(X_tr, z_tr)
        best_models[q] = qr
        joblib.dump(qr, models_dir / f"qr_fold{fold_no}_q{q:.2f}{postfix}.joblib")

    # ---------- Predictions and Metrics ----------

    z_lower = best_models[0.05].predict(X_te)
    z_med   = best_models[0.50].predict(X_te)
    z_upper = best_models[0.95].predict(X_te)

    y_lower = np.expm1(z_lower)
    y_med   = np.expm1(z_med)
    y_upper = np.expm1(z_upper)

    # Train metrics (log + original scale)
    z_tr_pred_full = best_models[0.50].predict(X_tr)
    train_mae_log = mean_absolute_error(z_tr, z_tr_pred_full)
    train_corr_log = spearman_r(z_tr, z_tr_pred_full)

    y_tr_pred_full = np.expm1(z_tr_pred_full)
    m_train_org = metrics_basic(y_tr, y_tr_pred_full)
    train_corr_org = spearman_r(y_tr, y_tr_pred_full)

    # Outer validation metrics
    val_true_log_arr = np.array(val_true_log_all)
    val_pred_log_arr = np.array(val_pred_log_all)

    val_mae_log = mean_absolute_error(val_true_log_arr, val_pred_log_arr)
    val_corr_log = spearman_r(val_true_log_arr, val_pred_log_arr)

    val_true_org = np.expm1(val_true_log_arr)
    val_pred_org = np.expm1(val_pred_log_arr)
    val_mae_org = mean_absolute_error(val_true_org, val_pred_org)
    val_corr_org = spearman_r(val_true_org, val_pred_org)

    # Test metrics
    test_mae_log = mean_absolute_error(np.log1p(y_te), z_med)
    test_corr_log = spearman_r(np.log1p(y_te), z_med)
    m_test_org = metrics_basic(y_te, y_med)
    test_corr_org = spearman_r(y_te, y_med)

    # Interval metrics
    cov_10_90 = np.mean((y_te >= y_lower) & (y_te <= y_upper))
    mean_width = np.mean(y_upper - y_lower)

    # Save test predictions per fold
    pd.DataFrame({
        "index": te_idx,
        "y_true": y_te,
        "y_pred_median": y_med,
        "pi_lower_005": y_lower,
        "pi_upper_095": y_upper
    }).to_csv(preds_dir / f"qr_test_preds_fold{fold_no}{postfix}.csv", index=False)

    # Append summary row
    all_rows.append({
        "fold": fold_no,
        "train_mae_org": m_train_org["mae"],
        "val_mae_org": val_mae_org,
        "test_mae_org": m_test_org["mae"],
        "train_corr_org": train_corr_org,
        "val_corr_org": val_corr_org,
        "test_corr_org": test_corr_org,
        "test_rmse_org": m_test_org["rmse"],
        "test_cov_10_90": cov_10_90,
        "pi_mean_width": mean_width
    })

    oof_preds[te_idx] = y_med
    oof_true[te_idx]  = y_te

# ---------------- Save Aggregated Results ----------------

df_all = pd.DataFrame(all_rows).sort_values("fold")
df_all.to_csv(metrics_dir / f"qr_nestedcv_metrics{postfix}.csv", index=False)

pd.DataFrame({
    "index": np.arange(len(y)),
    "y_true": oof_true,
    "y_pred": oof_preds
}).to_csv(preds_dir / f"qr_oof_preds{postfix}.csv", index=False)

print("\n>>> Quantile Regression nested CV complete.")

# PHASE C /B/Regressor Nested Cross-Validation with NGBoost for Probabilistic Regression

This script implements a **nested cross-validation (CV) pipeline** for training and evaluating **NGBoost** (Natural Gradient Boosting) models to predict continuous outcomes (e.g., Length of Stay) with **predictive uncertainty**.

NGBoost extends traditional gradient boosting by fitting a **full conditional probability distribution** rather than a single point estimate, allowing for *prediction intervals* and uncertainty quantification. [oai_citation:0‡Stanford Machine Learning Group](https://stanfordmlgroup.github.io/projects/ngboost/?utm_source=chatgpt.com)

---

### Overview of the Approach

1. **Input and Setup**

   - Imports standard Python libraries (NumPy, pandas, SciPy) and ML tools (StratifiedKFold, DecisionTreeRegressor, NGBoost).  
   - Defines helper functions for regression metrics including MAE, RMSE, R², Pearson *r*, and Spearman *r*.

2. **Configuration**

   - Sets a random seed (`RNG = 42`) for reproducibility.
   - Defines three target quantiles (`0.025`, `0.50`, `0.975`) to capture 95% prediction intervals.
   - Specifies directories for saved CV fold indices, trained models, evaluation metrics, and predictions.
   - Defines an inner-loop hyperparameter grid for learning rate, number of estimators, and tree depth.

3. **Nested Cross-Validation Loop**

   - Iterates over **outer CV folds** using saved indices.
   - For each fold:
     - Extracts training and test sets from indices.
     - Applies a log1p transform to the target to stabilise variance.
     - Uses an **inner StratifiedKFold** on the training set to tune hyperparameters based on median absolute error in log space.
     - Fits an NGBoost model using a DecisionTreeRegressor as the base learner and the Normal distribution.
     - Saves inner CV performance logs.

4. **Final Model Training and Evaluation**

   - Trains the selected NGBoost model on the full outer training set.
   - Computes predictions and predictive distributions on the test set.
     - Calculates median predictions and 95% prediction intervals.
   - Evaluates performance on training and test data using error, correlation, and interval coverage metrics.
   - Saves:
     - Per-fold test predictions with prediction interval bounds.
     - Out-of-fold (OOF) point predictions.
     - Trained NGBoost models.

---

### Performance and Outputs

The script reports:

- **Training and test performance** (MAE, RMSE, R², Pearson and Spearman correlation).
- **Prediction interval quality** (coverage and average interval width).
- **Per-fold and aggregated metrics** saved in structured CSV format.
- **Model artefacts** (serialized with `joblib`).

In [ ]:
"""
NGBoost Regression with Log1p Target Transform and 95% Prediction Intervals
- Uses saved fold indices: fold{n}_idx.npz containing train_sub_idx, val_idx, test_idx
- Hyperparameter tuning on validation set (log scale)
- Evaluates point metrics and prediction-interval quality on test set
- Saves models, per-fold predictions, and CV summary

Expected in memory before running:
    X : np.ndarray (n_samples, n_features)
    y : np.ndarray (n_samples,)
"""

import os, re, glob, itertools, joblib, json
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from ngboost import NGBRegressor
from ngboost.distns import Normal

# ------------------
# Helper functions
# ------------------
def rmse(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

def metrics(y_true, y_pred):
    return {
        "mae": mean_absolute_error(y_true, y_pred),
        "rmse": rmse(y_true, y_pred),
        "r2": r2_score(y_true, y_pred),
    }

# ------------------
# Config / Paths
# ------------------
RNG       = 42
QUANTILES = [0.025, 0.50, 0.975]  # 95% PI

base        = Path("/Users/daneshm/Documents/Kp_KAIMRC")
folds_dir   = base / "saved_folds"
models_dir  = base / "widthsaved_models"
metrics_dir = base / "widthmetrics_ngb_log1p"
preds_dir   = base / "widthpreds_ngb_log1p"

for d in (models_dir, metrics_dir, preds_dir):
    d.mkdir(parents=True, exist_ok=True)

# ------------------
# Load folds
# ------------------
fold_files = sorted(
    glob.glob(str(folds_dir / "fold*_idx.npz")),
    key=lambda p: int(re.search(r"fold(\d+)_idx\.npz", os.path.basename(p)).group(1))
)

all_rows = []

# ===================
# Cross-validation
# ===================
for fold_no, npz_path in enumerate(fold_files, start=1):
    print(f"\n=== Fold {fold_no}/{len(fold_files)} ===")

    idx = np.load(npz_path, allow_pickle=True)
    te_idx     = idx["test_idx"]
    tr_sub_idx = idx["train_sub_idx"]
    val_idx    = idx["val_idx"]

    X_tr_sub, y_tr_sub = X[tr_sub_idx], y[tr_sub_idx].astype(float)
    X_val,    y_val    = X[val_idx],    y[val_idx].astype(float)
    X_te,     y_te     = X[te_idx],     y[te_idx].astype(float)

    z_tr_sub = np.log1p(y_tr_sub)
    z_val    = np.log1p(y_val)

    # Hyper-parameter grid
    grid = {
        "learning_rate": [0.04],
        "n_estimators":  [100, 150],
        "max_depth":     [2, 3],
    }

    # Validation tuning (log scale)
    best_mae, best_model, best_params = np.inf, None, None
    for lr, n_est, md in itertools.product(grid["learning_rate"],
                                          grid["n_estimators"],
                                          grid["max_depth"]):

        base_tree = DecisionTreeRegressor(max_depth=md, random_state=RNG)
        ngb = NGBRegressor(
            Dist=Normal,
            Base=base_tree,
            learning_rate=lr,
            n_estimators=n_est,
            natural_gradient=True,
            verbose=False,
            random_state=RNG,
        )

        ngb.fit(X_tr_sub, z_tr_sub)
        z_val_pred = ngb.predict(X_val)
        mae_val_log = mean_absolute_error(z_val, z_val_pred)

        if mae_val_log < best_mae:
            best_mae = mae_val_log
            best_model = ngb
            best_params = {"learning_rate": lr, "n_estimators": n_est, "max_depth": md}

    print("Best params:", best_params)

    # ------------------
    # Test predictions
    # ------------------
    dist_te  = best_model.pred_dist(X_te)
    q_lower  = np.expm1(dist_te.ppf(QUANTILES[0]))
    q_median = np.expm1(dist_te.ppf(QUANTILES[1]))
    q_upper  = np.expm1(dist_te.ppf(QUANTILES[2]))
    y_te_pred = q_median

    # Point metrics
    m_org = metrics(y_te, y_te_pred)

    # Interval metrics
    cov_95  = np.mean((y_te >= q_lower) & (y_te <= q_upper))
    widths  = q_upper - q_lower
    width_abs = np.mean(widths)
    data_range = y_te.max() - y_te.min()
    width_norm = width_abs / data_range if data_range > 0 else np.nan
    sd_test = np.std(y_te, ddof=0)
    width_sd_ratio = width_abs / sd_test if sd_test > 0 else np.nan

    print(f"Test MAE={m_org['mae']:.4f} | RMSE={m_org['rmse']:.4f} | R2={m_org['r2']:.4f}")
    print(f"95% PI coverage = {cov_95:.3f}")
    print(f"Mean PI width = {width_abs:.3f} | Width/DataRange = {width_norm:.3f} | Width/SD = {width_sd_ratio:.3f}")

    # Save predictions
    pd.DataFrame({
        "y_true": y_te,
        "y_pred_median": y_te_pred,
        "pi_lower_025": q_lower,
        "pi_upper_975": q_upper,
    }).to_csv(preds_dir / f"ngb_log1p_test_preds_fold{fold_no}.csv", index=False)

    # Save metrics
    all_rows.append({
        "fold": fold_no,
        **best_params,
        "test_mae": m_org["mae"],
        "test_rmse": m_org["rmse"],
        "test_r2": m_org["r2"],
        "pi_95_coverage": cov_95,
        "pi_mean_width": width_abs,
        "pi_width_norm": width_norm,
        "pi_width_sd_ratio": width_sd_ratio,
    })

    # Save model
    joblib.dump(best_model, models_dir / f"ngb_log1p_fold{fold_no}.joblib")

# ------------------
# Save CV summary
# ------------------
df = pd.DataFrame(all_rows).sort_values("fold")
df.to_csv(metrics_dir / "ngb_log1p_cv_with_intervals.csv", index=False)

print("\n=== CV summary ===")
print(df)

In [ ]:
# PHASE D /B/Regressor for NGBoost Models

This script generates SHAP explanations for NGBoost models trained with your cross-validation pipeline.

**Features:**
- Works with saved NGBoost models (`ngb_log1p_fold{n}.joblib`)
- Uses the original fold indices (`fold{n}_idx.npz`) to slice the same `X` used during training
- Explains the *mean prediction on log1p scale* (suitable for ranking features)
- Outputs per-fold SHAP summary plots (beeswarm & bar) and CSVs
- Aggregates per-feature importances across all folds

**Requirements (in memory before running):**
- `X`: full feature matrix used during training
- `feature_names`: list/array of feature names of length `X.shape[1]`
- `postfix`: string matching your training script postfix (e.g., `"_exp1"`)